<a href="https://colab.research.google.com/github/ayagup/stablediffusion/blob/main/image_to_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

maygup123_dataset1_path = kagglehub.dataset_download('maygup123/dataset1')
maygup123_data12_path = kagglehub.dataset_download('maygup123/data12')

print('Data source import complete.')


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%pip install torch>=2.0.0 torchvision>=0.15.0 diffusers>=0.25.0 transformers>=4.35.0 accelerate>=0.24.0 Pillow>=10.0.0 opencv-python>=4.8.0 'numpy<2.0.0' xformers>=0.0.22 safetensors>=0.4.0 tqdm>=4.66.0 --quiet --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
fastai 2.8.4 requires torch<2.9,>=1.10, but you have torch 2.9.0 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.9.0 which is incompatible.
gradio 5.38.1 r

In [6]:
"""
Simple Image-to-Image Pipeline
Transforms images using various HuggingFace models for tasks like:
- Style transfer
- Image enhancement
- Super resolution
- Image-to-image translation
- Artistic filters
"""

import torch
from PIL import Image
from pathlib import Path
from typing import Union, Optional, List
import warnings
import os
warnings.filterwarnings('ignore')

# Configure HuggingFace Hub for better reliability
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'  # Use faster transfer
os.environ['HF_HUB_DOWNLOAD_TIMEOUT'] = '300'  # 5 minute timeout


class ImageToImagePipeline:
    """
    Simple image-to-image transformation pipeline supporting multiple models.
    """

    SUPPORTED_MODELS = {
        # Stable Diffusion Image-to-Image
        'sd-img2img': 'runwayml/stable-diffusion-v1-5',
        'sd-2-img2img': 'stabilityai/stable-diffusion-2-1',

        # ControlNet Models
        'controlnet-canny': 'lllyasviel/sd-controlnet-canny',
        'controlnet-depth': 'lllyasviel/sd-controlnet-depth',
        'controlnet-hed': 'lllyasviel/sd-controlnet-hed',
        'controlnet-mlsd': 'lllyasviel/sd-controlnet-mlsd',
        'controlnet-normal': 'lllyasviel/sd-controlnet-normal',
        'controlnet-openpose': 'lllyasviel/sd-controlnet-openpose',
        'controlnet-scribble': 'lllyasviel/sd-controlnet-scribble',
        'controlnet-seg': 'lllyasviel/sd-controlnet-seg',

        # InstantID
        'instantid': 'InstantX/InstantID',

        # Pix2Pix
        'instruct-pix2pix': 'timbrooks/instruct-pix2pix',

        # Super Resolution
        'swinir': 'caidas/swin2SR-classical-sr-x2-64',

        # Style Transfer
        'cycle-gan': 'Norod78/cyclegan-monet2photo',
    }

    def __init__(
        self,
        model_name: str = 'sd-img2img',
        device: Optional[str] = None,
        torch_dtype: torch.dtype = torch.float16,
        enable_attention_slicing: bool = True,
        enable_cpu_offload: bool = False
    ):
        """
        Initialize the image-to-image pipeline.

        Args:
            model_name: Model identifier from SUPPORTED_MODELS
            device: Device to use ('cuda', 'cpu', or None for auto-detect)
            torch_dtype: Data type for model weights
            enable_attention_slicing: Enable memory-efficient attention
            enable_cpu_offload: Enable model CPU offload for lower VRAM usage
        """
        self.model_name = model_name
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.torch_dtype = torch_dtype if self.device == 'cuda' else torch.float32

        print(f"🚀 Initializing Image-to-Image Pipeline")
        print(f"📦 Model: {model_name}")
        print(f"💻 Device: {self.device}")
        print(f"🔢 Dtype: {self.torch_dtype}")

        # Load appropriate pipeline
        self.pipeline = self._load_pipeline(
            model_name,
            enable_attention_slicing,
            enable_cpu_offload
        )

        print("✅ Pipeline loaded successfully!\n")

    def _load_pipeline(self, model_name: str, attention_slicing: bool, cpu_offload: bool):
        """Load the appropriate pipeline based on model type."""
        from diffusers import (
            StableDiffusionImg2ImgPipeline,
            StableDiffusionControlNetPipeline,
            StableDiffusionInstructPix2PixPipeline,
            ControlNetModel,
        )
        import time

        if model_name not in self.SUPPORTED_MODELS:
            raise ValueError(
                f"Unknown model: {model_name}. "
                f"Supported: {list(self.SUPPORTED_MODELS.keys())}"
            )

        model_id = self.SUPPORTED_MODELS[model_name]

        # Retry logic for model download
        max_retries = 3
        retry_delay = 10  # seconds

        # Stable Diffusion Img2Img
        if model_name in ['sd-img2img', 'sd-2-img2img']:
            for attempt in range(max_retries):
                try:
                    print(f"📥 Downloading model (attempt {attempt + 1}/{max_retries})...")
                    pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
                        model_id,
                        torch_dtype=self.torch_dtype,
                        safety_checker=None,
                        resume_download=True,
                        local_files_only=False,
                    )
                    break
                except Exception as e:
                    if attempt < max_retries - 1:
                        print(f"⚠️  Download failed: {e}")
                        print(f"🔄 Retrying in {retry_delay} seconds...")
                        time.sleep(retry_delay)
                        retry_delay *= 2  # Exponential backoff
                    else:
                        raise Exception(f"Failed to download model after {max_retries} attempts: {e}")

        # ControlNet Models
        elif model_name.startswith('controlnet-'):
            controlnet = ControlNetModel.from_pretrained(
                model_id,
                torch_dtype=self.torch_dtype
            )
            pipeline = StableDiffusionControlNetPipeline.from_pretrained(
                "runwayml/stable-diffusion-v1-5",
                controlnet=controlnet,
                torch_dtype=self.torch_dtype,
                safety_checker=None,
            )

        # InstructPix2Pix
        elif model_name == 'instruct-pix2pix':
            pipeline = StableDiffusionInstructPix2PixPipeline.from_pretrained(
                model_id,
                torch_dtype=self.torch_dtype,
                safety_checker=None,
            )

        else:
            # Default to img2img pipeline
            pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
                model_id,
                torch_dtype=self.torch_dtype,
                safety_checker=None,
            )

        pipeline = pipeline.to(self.device)

        # Memory optimizations
        if attention_slicing:
            pipeline.enable_attention_slicing()

        if cpu_offload and self.device == 'cuda':
            pipeline.enable_model_cpu_offload()

        return pipeline

    def transform(
        self,
        image: Union[str, Path, Image.Image],
        prompt: str,
        negative_prompt: Optional[str] = None,
        strength: float = 0.75,
        guidance_scale: float = 7.5,
        num_inference_steps: int = 50,
        num_images: int = 1,
        seed: Optional[int] = None,
    ) -> List[Image.Image]:
        """
        Transform an image based on a text prompt.

        Args:
            image: Input image (path or PIL Image)
            prompt: Text description of desired transformation
            negative_prompt: What to avoid in the output
            strength: How much to transform (0.0-1.0, higher = more change)
            guidance_scale: How closely to follow prompt (1.0-20.0)
            num_inference_steps: Number of denoising steps (higher = better quality)
            num_images: Number of variations to generate
            seed: Random seed for reproducibility

        Returns:
            List of transformed PIL Images
        """
        # Load image
        if isinstance(image, (str, Path)):
            image = Image.open(image).convert('RGB')

        # Set seed for reproducibility
        if seed is not None:
            generator = torch.Generator(device=self.device).manual_seed(seed)
        else:
            generator = None

        print(f"🎨 Transforming image...")
        print(f"📝 Prompt: {prompt}")
        print(f"💪 Strength: {strength}")
        print(f"🎯 Guidance: {guidance_scale}")

        # Generate
        with torch.inference_mode():
            if self.model_name == 'instruct-pix2pix':
                # InstructPix2Pix has different parameters
                output = self.pipeline(
                    prompt=prompt,
                    image=image,
                    num_inference_steps=num_inference_steps,
                    image_guidance_scale=guidance_scale,
                    guidance_scale=guidance_scale,
                    num_images_per_prompt=num_images,
                    generator=generator,
                )
            else:
                # Standard img2img pipeline
                output = self.pipeline(
                    prompt=prompt,
                    image=image,
                    negative_prompt=negative_prompt,
                    strength=strength,
                    guidance_scale=guidance_scale,
                    num_inference_steps=num_inference_steps,
                    num_images_per_prompt=num_images,
                    generator=generator,
                )

        print(f"✅ Generated {len(output.images)} image(s)\n")
        return output.images


def transform_image(
    image_path: Union[str, Path],
    prompt: str,
    output_dir: Union[str, Path] = "./output",
    model_name: str = 'sd-img2img',
    negative_prompt: Optional[str] = None,
    strength: float = 0.75,
    guidance_scale: float = 7.5,
    num_inference_steps: int = 50,
    num_images: int = 1,
    seed: Optional[int] = None,
) -> List[str]:
    """
    Simple function to transform an image and save results.

    Args:
        image_path: Path to input image
        prompt: Text description of desired transformation
        output_dir: Directory to save outputs
        model_name: Model to use (see SUPPORTED_MODELS)
        negative_prompt: What to avoid
        strength: Transformation strength (0.0-1.0)
        guidance_scale: Prompt adherence (1.0-20.0)
        num_inference_steps: Quality vs speed tradeoff
        num_images: Number of variations
        seed: Random seed

    Returns:
        List of output image paths
    """
    # Create output directory
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Initialize pipeline
    pipeline = ImageToImagePipeline(model_name=model_name)

    # Transform image
    images = pipeline.transform(
        image=image_path,
        prompt=prompt,
        negative_prompt=negative_prompt,
        strength=strength,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        num_images=num_images,
        seed=seed,
    )

    # Save results
    output_paths = []
    input_name = Path(image_path).stem

    for i, img in enumerate(images):
        if num_images == 1:
            output_path = output_dir / f"{input_name}_transformed.png"
        else:
            output_path = output_dir / f"{input_name}_transformed_{i+1}.png"

        img.save(output_path)
        output_paths.append(str(output_path))
        print(f"💾 Saved: {output_path}")

    return output_paths



In [8]:
# Example usage
# if __name__ == "__main__":
# Example 1: Basic style transformation
# print("=" * 60)
# print("Example 1: Style Transformation")
# print("=" * 60)

# transform_image(
#     image_path="/content/google-imagen-lead-image.jpeg",
#     prompt="transform into a beautiful oil painting in the style of Van Gogh, vibrant colors, impressionist",
#     negative_prompt="blurry, low quality, distorted",
#     model_name='sd-img2img',
#     strength=0.75,
#     guidance_scale=7.5,
#     num_inference_steps=50,
#     seed=42,
# )

# print("\n" + "=" * 60)
# print("Example 2: Image Enhancement")
# print("=" * 60)

# # Example 2: Enhance photo quality
# transform_image(
#     image_path="/content/google-imagen-lead-image.jpeg",
#     prompt="professional high quality photograph, sharp details, perfect lighting, 8k uhd",
#     negative_prompt="blurry, low quality, noisy, artifacts",
#     strength=0.3,  # Lower strength for subtle changes
#     guidance_scale=7.5,
#     num_inference_steps=50,
#     seed=42,
# )

# print("\n" + "=" * 60)
# print("Example 3: Artistic Transformation")
# print("=" * 60)

# # Example 3: Turn into anime style
# transform_image(
#     image_path="/content/google-imagen-lead-image.jpeg",
#     prompt="anime style illustration, studio ghibli, vibrant colors, detailed",
#     negative_prompt="realistic, photograph, blurry",
#     strength=0.85,  # Higher strength for dramatic change
#     guidance_scale=8.0,
#     num_inference_steps=50,
#     num_images=3,  # Generate 3 variations
#     seed=42,
# )

print("\n" + "=" * 60)
print("Example 4: Using InstructPix2Pix")
print("=" * 60)

# Example 4: Use natural language instructions
transform_image(
    image_path="/content/google-imagen-lead-image_transformed (1).png",
    prompt="a bird sitting on a table in a snowy  weather. Winter with snow everywhere",
    negative_prompt="bad quality, low resolution, low quality, unclear",
    model_name='instruct-pix2pix',
    guidance_scale=7.5,
    num_inference_steps=50,
    seed=42,
)



Example 4: Using InstructPix2Pix
🚀 Initializing Image-to-Image Pipeline
📦 Model: instruct-pix2pix
💻 Device: cuda
🔢 Dtype: torch.float16


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

✅ Pipeline loaded successfully!

🎨 Transforming image...
📝 Prompt: a bird sitting on a table in a snowy  weather. Winter with snow everywhere
💪 Strength: 0.75
🎯 Guidance: 7.5


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Generated 1 image(s)

💾 Saved: output/google-imagen-lead-image_transformed (1)_transformed.png


['output/google-imagen-lead-image_transformed (1)_transformed.png']

In [4]:
1!ls output/

google-imagen-lead-image_transformed.png
